In [1]:
# FOR GOOGLE COLAB ONLY
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from math import sqrt

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.config.experimental.list_physical_devices('GPU')

[]

In [20]:
# FOR GOOGLE COLAB ONLY
DRIVE_PATH  = "/content/drive/MyDrive/Colab Notebooks/supervised/"

In [3]:
# FOR LOCALHOST
DRIVE_PATH = ""

In [4]:
# Callback function: adjust learning rate to 1e-05 when epochs >= 100
# Input: epoch: int
# Return: rate: float
def adjust_lr(epoch):
    rate = 0.0001
    if epoch >= 100:
        rate /= 10
    return rate

In [5]:
# Generate the output directory for the model
# Input: d: dict
# Return: return_d: dict
def make_output_dir(d):
    tgt_path = "/%s/e%d_t%d_b_%d_n%d" % (d["country"], d["epoch"], d["timestep"], d["batch"], d["neurons"])
    country_path = "/%s" % d["country"]
    return_d = {}
    print(tgt_path)
    dirs = ["models", "pred_graphs", "loss_graphs"]
    for dir in dirs:
        c_pth = DRIVE_PATH + dir + country_path
        t_pth = DRIVE_PATH + dir + tgt_path
        if not os.path.exists(c_pth):
            os.makedirs(c_pth)
        return_d[dir] = t_pth
    return return_d

In [6]:
# Revert the scaled data to the orginal data
# Input: x: np.ndarray
#           y: np.ndarray
#           scaler: sklearn.preprocess.MinMaxScaler
# Return: x_unscaled: np.ndarray
#             y_unscaled: np.ndarray
def unscale(x, y, scaler):
    data = np.concatenate((x, y), axis=1)
    unscaled_data = scaler.inverse_transform(data)
    x_unscaled = unscaled_data[:,:-1]
    y_unscaled = unscaled_data[:,-1]
    return x_unscaled, y_unscaled

In [7]:
# Train the model by the input parameters
# Input: features, timestep, train_epochs, batch_s: int
#            country: str
# Return:  (train_loss, test_loss) -> (float, float)
def train_by_country(features, timestep, train_epochs, batch_s, neurons, country):
    path_dict = {"epoch": train_epochs, "batch": batch_s, "country": country, "timestep": timestep, "neurons": neurons}
    dirs = make_output_dir(path_dict)

    # load dataset, scale and split
    dataset = pd.read_csv(DRIVE_PATH + "country_csv/supervised_%d/%s_supervised.csv" % (timestep, country), index_col=0, header=0)
    dataset.fillna(0)
    scaler = MinMaxScaler(feature_range=(0,1))
    values = scaler.fit_transform(dataset.values)
    X = values[:,:-1]
    y = values[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train_r = X_train.reshape((X_train.shape[0], timestep, features))
    X_test_r = X_test.reshape((X_test.shape[0], timestep, features))

    # build up the model
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(X_train_r.shape[1], X_train_r.shape[2])))
    model.add(Dense(neurons))
    model.add(Dense(1))
    opt = Adam(lr=0.0001)
    model.compile(loss="mse", optimizer=opt)

    # set up callback function for learning rate scheduler
    lrate = LearningRateScheduler(adjust_lr)
    callbacks_list = [lrate]

    # fit network
    print("Training:%s" %country)
    history = model.fit(X_train_r, y_train, epochs=train_epochs, batch_size=batch_s, validation_data=(X_test_r, y_test), verbose=0, shuffle=False, callbacks = callbacks_list)

    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    # save to directory
    pyplot.savefig(dirs["loss_graphs"])
    # pyplot.show()
    pyplot.clf()

    predicted = model.predict(X_test_r)
    unscaled_seq, unscaled_pred = unscale(X_test, predicted, scaler=scaler)
    unscaled_seq, unscaled_test = unscale(X_test, y_test.reshape(-1,1), scaler=scaler)
    pyplot.plot(unscaled_pred, label='predicted')
    pyplot.plot(unscaled_test, label="actual")
    pyplot.legend()
    # save to directory
    pyplot.savefig(dirs["pred_graphs"])
    # pyplot.show()
    pyplot.clf()
    model.save(dirs["models"])
    # return training loss
    return sum(history.history["loss"]) / len(history.history["loss"]), sum(history.history["val_loss"]) / len(history.history["val_loss"]), 

In [8]:
# single country test
country = "GBR"
train_by_country(features = 43, timestep = 1, train_epochs = 500, batch_s = 3, neurons = 200, country=country)

/FRA/e10_t1_b_3_n200
Training:FRA
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/FRA/e10_t1_b_3_n200\assets


(0.0019773114938288925, 0.052754147723317144)

<Figure size 432x288 with 0 Axes>

In [9]:
# batch execution
features = 43
timestep = 1
train_losses = []
test_losses = []
# train a model for each country that we have
for root, dir, files in os.walk(DRIVE_PATH + "country_csv/supervised_%d/" % timestep):
    for file in tqdm(sorted(files)):
        train_loss, test_loss = train_by_country(features = 43, timestep = 1, train_epochs = 500, batch_s = 3, neurons = 200, country=file)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print(train_losses, test_losses)

  0%|          | 0/37 [00:00<?, ?it/s]

/SGP/e500_t1_b_3_n200
Training:SGP
(63, 43) (63, 1)
(63, 44)
35.47024679929018
(63, 43) (63, 1)
(63, 44)
23.0
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/supervised/models/SGP/e500_t1_b_3_n200/assets


  3%|▎         | 1/37 [02:14<1:20:49, 134.72s/it]

[0.007038058820180595, 0.001692266707541421, 0.0002592144685040694, 0.002397439305204898, 0.002869029426481575, 0.0024772747105453164, 0.007205062361434102, 0.00013566622945654672, 0.00015732715976628242, 0.007558043436147272, 0.00011517629760783166, 0.00540777232311666, 0.004690073286183178, 0.004278627061285079, 0.00012055437093658838, 0.009132190885022283, 0.004155155834741891, 0.001973639988573268, 0.0011195111217675731, 0.004326898943632841, 0.008042383526451886, 0.016242972910404205, 0.014226134518161416, 0.003165411848574877, 0.007964463587850332, 0.0010386412066873162, 0.01684532544761896, 0.0011093914762604982, 6.219137770676753e-05, 0.005149696069769561, 0.0032942376961000264, 0.009617824671790004, 0.007786353777162731, 0.006820476376451552, 0.009894220126792789, 0.005431790510192514, 0.016134319499135016, 0.009427634542807936, 0.00488838560692966, 0.00973883163370192, 0.00032924873701995237, 0.00021932110789930448, 0.0002502465540310368, 0.012970477320253849, 0.0050809344938

  5%|▌         | 2/37 [04:16<1:16:24, 130.99s/it]

...

 97%|█████████▋| 36/37 [1:11:21<01:58, 118.09s/it]

[0.007038058820180595, 0.001692266707541421, 0.0002592144685040694, 0.002397439305204898, 0.002869029426481575, 0.0024772747105453164, 0.007205062361434102, 0.00013566622945654672, 0.00015732715976628242, 0.007558043436147272, 0.00011517629760783166, 0.00540777232311666, 0.004690073286183178, 0.004278627061285079, 0.00012055437093658838, 0.009132190885022283, 0.004155155834741891, 0.001973639988573268, 0.0011195111217675731, 0.004326898943632841, 0.008042383526451886, 0.016242972910404205, 0.014226134518161416, 0.003165411848574877, 0.007964463587850332, 0.0010386412066873162, 0.01684532544761896, 0.0011093914762604982, 6.219137770676753e-05, 0.005149696069769561, 0.0032942376961000264, 0.009617824671790004, 0.007786353777162731, 0.006820476376451552, 0.009894220126792789, 0.005431790510192514, 0.016134319499135016, 0.009427634542807936, 0.00488838560692966, 0.00973883163370192, 0.00032924873701995237, 0.00021932110789930448, 0.0002502465540310368, 0.012970477320253849, 0.0050809344938

100%|██████████| 37/37 [1:13:18<00:00, 118.87s/it]

[0.007038058820180595, 0.001692266707541421, 0.0002592144685040694, 0.002397439305204898, 0.002869029426481575, 0.0024772747105453164, 0.007205062361434102, 0.00013566622945654672, 0.00015732715976628242, 0.007558043436147272, 0.00011517629760783166, 0.00540777232311666, 0.004690073286183178, 0.004278627061285079, 0.00012055437093658838, 0.009132190885022283, 0.004155155834741891, 0.001973639988573268, 0.0011195111217675731, 0.004326898943632841, 0.008042383526451886, 0.016242972910404205, 0.014226134518161416, 0.003165411848574877, 0.007964463587850332, 0.0010386412066873162, 0.01684532544761896, 0.0011093914762604982, 6.219137770676753e-05, 0.005149696069769561, 0.0032942376961000264, 0.009617824671790004, 0.007786353777162731, 0.006820476376451552, 0.009894220126792789, 0.005431790510192514, 0.016134319499135016, 0.009427634542807936, 0.00488838560692966, 0.00973883163370192, 0.00032924873701995237, 0.00021932110789930448, 0.0002502465540310368, 0.012970477320253849, 0.0050809344938

<Figure size 432x288 with 0 Axes>

In [10]:
# FOR BATCH EXECUTION
print(sum(train_losses) / len(train_losses)) # Average train losses
print(sum(test_losses) / len(test_losses))   # Average test losses

0.005264751757801552
0.049185052510442896
